In [2]:
# === CELL 1: Mount Drive and Setup ===
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content')

# Clone repo if not already there
if not os.path.exists('fednams-plus'):
    !git clone https://github.com/siddharth10ss/fednams-plus.git

%cd fednams-plus
!pip install -q torch torchvision pandas numpy pillow

print("✓ Setup complete")

Mounted at /content/drive
Cloning into 'fednams-plus'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 32 (delta 0), reused 32 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 26.26 KiB | 8.75 MiB/s, done.
/content/fednams-plus
✓ Setup complete


In [ ]:
# ============================================================================
# OPTION 3: Extract Subset of NIH Dataset (10,000 images)
# ============================================================================

# === CELL 1: Clean up partial extraction ===
import shutil
from pathlib import Path

# First, clean up any partial extraction
partial_dirs = [
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_001',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_002',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_003',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_004',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_005',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_006',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_007',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_008',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_009',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_010',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_011',
    '/content/drive/MyDrive/NIH Chest XRAY Dataset/images_012',
]

print("🧹 Cleaning up partial extraction...")
for dir_path in partial_dirs:
    if Path(dir_path).exists():
        print(f"  Removing {Path(dir_path).name}...")
        shutil.rmtree(dir_path)

print("✓ Cleanup complete")

🧹 Cleaning up partial extraction...
  Removing images_006...
  Removing images_007...
  Removing images_008...


In [3]:
# === CELL 2: Extract Subset ===
import zipfile
from pathlib import Path

zip_path = '/content/drive/MyDrive/NIH Chest XRAY Dataset/archive.zip'
extract_dir = '/content/drive/MyDrive/NIH Chest XRAY Dataset/'

print("📦 Extracting subset of NIH dataset...")
print("📊 Target: 10,000 images + CSV files")
print("⏳ This will take 5-10 minutes...\n")

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    all_files = zip_ref.namelist()

    # Separate files by type
    csv_files = [f for f in all_files if f.endswith('.csv')]
    pdf_files = [f for f in all_files if f.endswith('.pdf')]
    image_files = [f for f in all_files if f.endswith('.png')]

    print(f"📁 Found in archive:")
    print(f"  - CSV files: {len(csv_files)}")
    print(f"  - PDF files: {len(pdf_files)}")
    print(f"  - Image files: {len(image_files)}")

    # Select files to extract
    files_to_extract = []
    files_to_extract.extend(csv_files)  # All CSV files
    files_to_extract.extend(pdf_files)  # All PDF files
    files_to_extract.extend(image_files[:10000])  # First 10,000 images

    print(f"\n📦 Extracting {len(files_to_extract)} files...")
    print(f"  - CSV/PDF: {len(csv_files) + len(pdf_files)}")
    print(f"  - Images: 10,000")

    # Extract with progress
    for i, file in enumerate(files_to_extract, 1):
        zip_ref.extract(file, extract_dir)

        if i % 500 == 0:
            progress = (i / len(files_to_extract)) * 100
            print(f"  Progress: {i}/{len(files_to_extract)} ({progress:.1f}%)")

print("\n✓ Extraction complete!")


📦 Extracting subset of NIH dataset...
📊 Target: 10,000 images + CSV files
⏳ This will take 5-10 minutes...

📁 Found in archive:
  - CSV files: 2
  - PDF files: 4
  - Image files: 112120

📦 Extracting 10006 files...
  - CSV/PDF: 6
  - Images: 10,000
  Progress: 500/10006 (5.0%)
  Progress: 1000/10006 (10.0%)
  Progress: 1500/10006 (15.0%)
  Progress: 2000/10006 (20.0%)
  Progress: 2500/10006 (25.0%)
  Progress: 3000/10006 (30.0%)
  Progress: 3500/10006 (35.0%)
  Progress: 4000/10006 (40.0%)
  Progress: 4500/10006 (45.0%)
  Progress: 5000/10006 (50.0%)
  Progress: 5500/10006 (55.0%)
  Progress: 6000/10006 (60.0%)
  Progress: 6500/10006 (65.0%)
  Progress: 7000/10006 (70.0%)
  Progress: 7500/10006 (75.0%)
  Progress: 8000/10006 (80.0%)
  Progress: 8500/10006 (84.9%)
  Progress: 9000/10006 (89.9%)
  Progress: 9500/10006 (94.9%)
  Progress: 10000/10006 (99.9%)

✓ Extraction complete!


In [4]:
# === CELL 3: Verify Extraction ===
import pandas as pd

data_dir = Path('/content/drive/MyDrive/NIH Chest XRAY Dataset')

print("\n📁 Verifying extracted files...")

# Check CSV files
csv_files = list(data_dir.glob('*.csv'))
print(f"\n✓ CSV files: {len(csv_files)}")
for csv in csv_files:
    print(f"  - {csv.name}")

# Check image directories
image_dirs = [d for d in data_dir.glob('images*') if d.is_dir()]
print(f"\n✓ Image directories: {len(image_dirs)}")

# Count images
total_images = 0
for img_dir in image_dirs:
    count = len(list(img_dir.rglob('*.png')))
    total_images += count
    print(f"  - {img_dir.name}: {count} images")

print(f"\n✓ Total images extracted: {total_images}")

# Load labels
labels_file = data_dir / 'Data_Entry_2017.csv'
if labels_file.exists():
    labels_df = pd.read_csv(labels_file)
    print(f"\n✓ Labels file loaded: {len(labels_df)} total entries")
    print(f"  (We have images for first {total_images} entries)")

print("\n" + "="*60)
print("✅ SUBSET EXTRACTION SUCCESSFUL!")
print("="*60)
print(f"\n📊 What you have:")
print(f"  ✓ 10,000 chest X-ray images")
print(f"  ✓ Complete labels CSV (all 112k entries)")
print(f"  ✓ Bounding box CSV")
print(f"  ✓ README PDF")
print(f"\n💡 This is plenty for:")
print(f"  - Development and testing")
print(f"  - Training federated models")
print(f"  - Validating the complete pipeline")
print(f"\n📝 You can extract more images later if needed")



📁 Verifying extracted files...

✓ CSV files: 2
  - BBox_List_2017.csv
  - Data_Entry_2017.csv

✓ Image directories: 2
  - images_001: 4999 images
  - images_002: 5001 images

✓ Total images extracted: 10000

✓ Labels file loaded: 112120 total entries
  (We have images for first 10000 entries)

✅ SUBSET EXTRACTION SUCCESSFUL!

📊 What you have:
  ✓ 10,000 chest X-ray images
  ✓ Complete labels CSV (all 112k entries)
  ✓ Bounding box CSV
  ✓ README PDF

💡 This is plenty for:
  - Development and testing
  - Training federated models
  - Validating the complete pipeline

📝 You can extract more images later if needed


In [7]:
# === Pull latest code with data module ===
%cd /content/fednams-plus
!git pull origin main

print("\n📁 Checking data module...")
!ls -la data/

print("\n✓ Data module should now be available!")


/content/fednams-plus
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 2), reused 10 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (10/10), 10.07 KiB | 1.26 MiB/s, done.
From https://github.com/siddharth10ss/fednams-plus
 * branch            main       -> FETCH_HEAD
   ffd5871..43f24c0  main       -> origin/main
Updating ffd5871..43f24c0
Fast-forward
 .gitignore           |  18 +--
 data/README.md       | 135 ++++++++++++++++++++++
 data/__init__.py     |  18 +++
 data/base_dataset.py | 151 ++++++++++++++++++++++++
 data/downloader.py   | 164 ++++++++++++++++++++++++++
 data/partitioner.py  | 320 +++++++++++++++++++++++++++++++++++++++++++++++++++
 data/preprocessor.py | 246 +++++++++++++++++++++++++++++++++++++++
 7 files changed, 1045 insertions(+), 7 deletions(-)
 create mode 100644 data/README.md
 create mode 100644 data/__init__.py
 create mode 100644 data/base_da

In [9]:
# === Pull the fix and re-run ===
%cd /content/fednams-plus
!git pull origin main

# Re-import the fixed module
import importlib
import sys

# Remove old module from cache
if 'data.partitioner' in sys.modules:
    del sys.modules['data.partitioner']
if 'data' in sys.modules:
    del sys.modules['data']

# Re-import
from data import FederatedDataPartitioner
from pathlib import Path

print("✓ Updated module imported")

# Now generate statistics (this should work)
print("\n📊 Generating statistics...")

stats_dir = Path('/content/drive/MyDrive/NIH Chest XRAY Dataset/outputs/partitions')
stats = partitioner.generate_statistics(client_datasets, stats_dir)

print("✓ Statistics saved to Google Drive!")
print(f"\nPartition Statistics:")
print(stats)

# Test all strategies
print(f"\n🧪 Testing all partitioning strategies...")

strategies = {
    'dirichlet': {'alpha': 0.5},
    'pathology': {},
    'quantity': {}
}

for strategy_name, kwargs in strategies.items():
    print(f"\n--- {strategy_name.capitalize()} Strategy ---")

    part = FederatedDataPartitioner(
        num_clients=5,
        strategy=strategy_name,
        min_samples=200,
        **kwargs
    )

    clients = part.partition(dataset)

    for cid, subset in clients.items():
        print(f"  Client {cid}: {len(subset)} samples")

print("\n" + "="*70)
print("✅ TASK 2: DATA MODULE - FULLY VALIDATED!")
print("="*70)
print(f"\n📊 Complete Test Summary:")
print(f"  ✓ Dataset: NIH Chest X-rays")
print(f"  ✓ Images extracted: 10,000")
print(f"  ✓ Test dataset: 5,000 samples")
print(f"  ✓ Image size: 1024x1024 → 224x224")
print(f"  ✓ Classes: 15 (14 diseases + No findings)")
print(f"  ✓ Federated clients: 5")
print(f"\n✅ All Components Validated:")
print(f"  ✓ DataDownloader (manual upload)")
print(f"  ✓ DataPreprocessor (transforms, augmentation)")
print(f"  ✓ MIMICCXRDataset (lazy loading)")
print(f"  ✓ FederatedDataPartitioner (3 strategies)")
print(f"  ✓ Statistics generation (fixed!)")
print(f"\n📝 READY FOR TASK 3: BUILD NAM MODELS!")
print(f"💾 Data persists in Google Drive")
print(f"🎉 Task 2 Complete!")


/content/fednams-plus
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 437 bytes | 145.00 KiB/s, done.
From https://github.com/siddharth10ss/fednams-plus
 * branch            main       -> FETCH_HEAD
   43f24c0..8cb0e70  main       -> origin/main
Updating 43f24c0..8cb0e70
Fast-forward
 data/partitioner.py | 6 +++---
 1 file changed, 3 insertions(+), 3 deletions(-)
✓ Updated module imported

📊 Generating statistics...


TypeError: Object of type float32 is not JSON serializable

In [10]:
# === Workaround: Reload module without restart ===
%cd /content/fednams-plus
!git pull origin main

# Force reload
import sys
import importlib

# Remove all data module variants from cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('data')]
for mod in modules_to_remove:
    del sys.modules[mod]

# Re-import fresh
from data import FederatedDataPartitioner
from pathlib import Path

print("✓ Module reloaded with fix")

# Create NEW partitioner instance
partitioner_new = FederatedDataPartitioner(
    num_clients=5,
    strategy='dirichlet',
    alpha=0.5,
    min_samples=200
)

# Re-partition
client_datasets_new = partitioner_new.partition(dataset)

print("\n📊 Generating statistics with fixed code...")
stats_dir = Path('/content/drive/MyDrive/NIH Chest XRAY Dataset/outputs/partitions')
stats = partitioner_new.generate_statistics(client_datasets_new, stats_dir)

print("✓ Statistics saved!")
print(stats)

print("\n✅ TASK 2 COMPLETE!")


/content/fednams-plus
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 452 bytes | 64.00 KiB/s, done.
From https://github.com/siddharth10ss/fednams-plus
 * branch            main       -> FETCH_HEAD
   8cb0e70..2030c69  main       -> origin/main
Updating 8cb0e70..2030c69
Fast-forward
 data/partitioner.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)
✓ Module reloaded with fix

📊 Generating statistics with fixed code...
✓ Statistics saved!
   client_id  num_samples                                 label_distribution  \
0          0          383  [50.0, 21.0, 146.0, 266.0, 168.0, 204.0, 180.0...   
1          1          891  [606.0, 525.0, 471.0, 442.0, 446.0, 422.0, 434...   
2          2         2434  [1674.0, 1466.0, 1184.0, 1189.0, 1218.0, 1213....   
3          3          448  [27.0, 186.0, 79.

In [11]:
# === Test Task 3: Models Module ===
%cd /content/fednams-plus
!git pull origin main

import torch
from models import FedNAMsModel, FedAvgCNN, CentralizedNAM
from configs import ModelConfig

print("🧪 Testing Models Module...\n")

# Create config
config = ModelConfig(
    backbone='resnet18',
    pretrained=True,
    feature_dim=512,
    num_classes=15,  # NIH has 15 classes
    nam_hidden_units=[64, 32],
    dropout=0.3
)

# Test FedNAMsModel
print("1. Testing FedNAMsModel...")
model = FedNAMsModel(config)
print(f"✓ Model created")
print(f"  Parameters: {model.get_num_parameters():,}")

# Test forward pass
dummy_input = torch.randn(2, 3, 224, 224)
output = model(dummy_input)
print(f"✓ Forward pass: {dummy_input.shape} → {output.shape}")

# Test with contributions
pred, features, contributions = model.forward_with_contributions(dummy_input)
print(f"✓ With contributions:")
print(f"  Predictions: {pred.shape}")
print(f"  Features: {features.shape}")
print(f"  Contributions: {len(contributions)} feature NNs")

# Test parameter serialization
params = model.get_parameters()
print(f"✓ Parameter serialization: {len(params)} parameters")

# Test baselines
print("\n2. Testing FedAvgCNN...")
baseline = FedAvgCNN(config)
output = baseline(dummy_input)
print(f"✓ FedAvgCNN: {output.shape}")

print("\n3. Testing CentralizedNAM...")
centralized = CentralizedNAM(config)
output = centralized(dummy_input)
print(f"✓ CentralizedNAM: {output.shape}")

print("\n" + "="*60)
print("✅ TASK 3: MODELS MODULE - FULLY VALIDATED!")
print("="*60)
print("\n📝 Ready for Task 4: Training Module!")


/content/fednams-plus
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8 (delta 3), reused 8 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (8/8), 6.35 KiB | 433.00 KiB/s, done.
From https://github.com/siddharth10ss/fednams-plus
 * branch            main       -> FETCH_HEAD
   2030c69..f7bd8b7  main       -> origin/main
Updating 2030c69..f7bd8b7
Fast-forward
 models/__init__.py          |   6 +-
 models/baselines.py         | 236 ++++++++++++++++++++++++++++++++++++++++++++
 models/feature_extractor.py | 149 ++++++++++++++++++++++++++++
 models/fednams_model.py     | 153 ++++++++++++++++++++++++++++
 models/nam_head.py          | 203 +++++++++++++++++++++++++++++++++++++
 5 files changed, 745 insertions(+), 2 deletions(-)
 create mode 100644 models/baselines.py
 create mode 100644 models/feature_extractor.py
 create mode 100644 models/fednams_model.py
 create mode 100644 models/n

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 53.8MB/s]


✓ Model created
  Parameters: 12,331,614
✓ Forward pass: torch.Size([2, 3, 224, 224]) → torch.Size([2, 15])
✓ With contributions:
  Predictions: torch.Size([2, 15])
  Features: torch.Size([2, 512])
  Contributions: 512 feature NNs
✓ Parameter serialization: 3135 parameters

2. Testing FedAvgCNN...
✓ FedAvgCNN: torch.Size([2, 15])

3. Testing CentralizedNAM...
✓ CentralizedNAM: torch.Size([2, 15])

✅ TASK 3: MODELS MODULE - FULLY VALIDATED!

📝 Ready for Task 4: Training Module!
